### Libraries

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppresses INFO and WARNING logs

from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
import sys

import numpy as np
import pandas as pd
import cvxpy as cp
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

2025-05-05 12:18:19.214736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-05 12:18:19.244589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-05 12:18:19.275901: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [39]:
# Navigate up two directories from the notebook's location to reach the root
sys.path.append(os.path.abspath(os.path.join('..', '..')))
from utils.data_processor_v2 import DataProcessorUpdated

In [40]:
processor = DataProcessorUpdated(self_back_window = 24*7, prediction_window = 10)

In [41]:
processor.load_and_clean_data()

2023-12-31 23:55:00


/insomnia001/home/hl3837/research/code/utils/data_processor_v2.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[time_col] = pd.to_datetime(df[time_col])


2024-01-01 23:00:00
2023-12-31 23:55:00
2023-12-31 23:55:00


In [42]:
processor.combine_all_data()

In [43]:
processor.split_data(feature_columns=["delta_price"], target_col='delta_price')

In [44]:
type(processor.x_test_df_reg)

pandas.core.frame.DataFrame

In [45]:
processor.apply_gaussian_transform()

In [46]:
processor.shift_data()

In [51]:
x_train_mlp, x_val_mlp, x_test_mlp, y_train_mlp, y_val_mlp, y_test_mlp = processor.flatten()

In [48]:
df, x_train_mlp, x_val_mlp, x_test_mlp, y_train_mlp, y_val_mlp, y_test_mlp = processor.get_data()

In [52]:
(x_train_mlp.shape)

(313225, 2016)

In [53]:
from MLP import MLPModel
new_model = MLPModel(input_shape = x_train_mlp.shape[1], output_shape = y_train_mlp.shape[1], hidden_layers=[512, 512, 512])
new_model.build()
mlp_model = new_model.get_model()

/insomnia001/shared/apps/anaconda/2023.09/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
import tensorflow as tf

# Check available devices
print("Available devices:", tf.config.list_physical_devices())

# Check for GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("Using GPU")
    for gpu in gpus:
        print(f"GPU details: {gpu}")
else:
    print("No GPU detected, using CPU")


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
No GPU detected, using CPU


In [55]:
from utils.model_trainer import ModelTrainer

newTrainder = ModelTrainer(mlp_model, x_train_mlp, y_train_mlp, x_val_mlp, y_val_mlp)
newTrainder.train()

history = newTrainder.get_training_history()


Epoch 1/40
9789/9789 - 38s - 4ms/step - accuracy: 0.0204 - loss: 0.1973 - mae: 0.5159 - val_accuracy: 0.0173 - val_loss: 0.4750 - val_mae: 0.9279
Epoch 2/40
9789/9789 - 37s - 4ms/step - accuracy: 0.0278 - loss: 0.1149 - mae: 0.3629 - val_accuracy: 0.0164 - val_loss: 0.4910 - val_mae: 0.9474
Epoch 3/40
9789/9789 - 38s - 4ms/step - accuracy: 0.0329 - loss: 0.0920 - mae: 0.3113 - val_accuracy: 0.0179 - val_loss: 0.4938 - val_mae: 0.9534
Epoch 4/40
9789/9789 - 38s - 4ms/step - accuracy: 0.0374 - loss: 0.0818 - mae: 0.2856 - val_accuracy: 0.0180 - val_loss: 0.4972 - val_mae: 0.9583
Epoch 5/40
9789/9789 - 36s - 4ms/step - accuracy: 0.0410 - loss: 0.0761 - mae: 0.2708 - val_accuracy: 0.0177 - val_loss: 0.5021 - val_mae: 0.9654
Epoch 6/40
9789/9789 - 37s - 4ms/step - accuracy: 0.0434 - loss: 0.0724 - mae: 0.2613 - val_accuracy: 0.0176 - val_loss: 0.5034 - val_mae: 0.9673
Epoch 7/40
9789/9789 - 37s - 4ms/step - accuracy: 0.0455 - loss: 0.0697 - mae: 0.2545 - val_accuracy: 0.0174 - val_loss: 0.5

### Model Building

In [61]:
processor.x_test_df_reg.shape

(105120, 1)

In [59]:
x_test_mlp.shape

(102985, 2016)

In [56]:
#load model

# Make predictions with the MLP model
y_test_pred = mlp_model.predict(x_test_mlp)

# Evaluate Prediction
# Evaluate the model's prediction
accuray, loss, mae = newTrainder.model_evaluate(x_test_mlp, y_test_mlp)
print(accuray, loss, mae)

# Evaluate the model's prediction with custom eval
mae, rmse = newTrainder.custom_evaluate(y_test_pred, y_test_mlp)
print(mae, rmse)

3219/3219 ━━━━━━━━━━━━━━━━━━━━ 3s 847us/step
3219/3219 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0122 - loss: 0.4397 - mae: 0.8778
0.4310206472873688 0.8612468838691711 0.01224450208246708
MAE: 0.86
RMSE: 1.12


0.8612465281037693 1.117693969413983


In [62]:
y_pred_residual = processor.inverse_gaussian_transform_y(y_test_pred)

/insomnia001/shared/apps/anaconda/2023.09/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(
/insomnia001/shared/apps/anaconda/2023.09/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(
/insomnia001/shared/apps/anaconda/2023.09/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(
/insomnia001/shared/apps/anaconda/2023.09/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(
/insomnia001/shared/apps/anaconda/2023.09/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but Qu

In [63]:
y_pred_residual.shape

(102985, 120)

In [68]:
dap = pd.read_csv('../../datasset/hourly_ercot_day_ahead_sl_2019_2023.csv')
dap['timestamp'] = pd.to_datetime(dap['timestamp'])
dap.set_index('timestamp', inplace=True)
dap = dap.resample('5min').ffill()
date_range = pd.date_range(start=dap.index.min(), end='2024-01-01 23:55:00', freq='5min')
dap = dap[~dap.index.duplicated(keep='first')]
dap = dap.reindex(date_range, fill_value=np.nan)
dap.interpolate(method='time', inplace=True)

/tmp/ipykernel_2872676/1012585795.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dap['timestamp'] = pd.to_datetime(dap['timestamp'])


In [74]:
slice_length_diff = 12 * 24 * 365 * 4 + 7*24*12 - 1
look_ahead = 120
# prepare the nodal real-time price in the required shape
actual_dap = dap.iloc[slice_length_diff:-288]
actual_dap_rolled = pd.DataFrame([pd.Series(actual_dap['SystemLambda'].iloc[i+1:i+1+look_ahead].values) for i in range(len(actual_dap) - look_ahead)])
actual_dap_rolled.columns = [f'{i+1}' for i in range(look_ahead)]
actual_dap_rolled['sced_time_stamp_local'] = actual_dap.index[:len(actual_dap_rolled)].values
actual_dap_rolled = actual_dap_rolled[['sced_time_stamp_local'] + [col for col in actual_dap_rolled.columns if col != 'sced_time_stamp_local']]
actual_dap_rolled = actual_dap_rolled.iloc[:,:look_ahead+1]

In [75]:
actual_dap_rolled

,sced_time_stamp_local,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,2023-01-07 23:55:00,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,...,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088
1,2023-01-08 00:00:00,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,...,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,21.9528
2,2023-01-08 00:05:00,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,...,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,21.9528,21.9528
3,2023-01-08 00:10:00,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,...,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,21.9528,21.9528,21.9528
4,2023-01-08 00:15:00,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,27.1811,24.4058,...,23.7088,23.7088,23.7088,23.7088,23.7088,23.7088,21.9528,21.9528,21.9528,21.9528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102980,2023-12-31 13:35:00,12.5549,12.5549,12.5549,12.5549,12.1478,12.1478,12.1478,12.1478,12.1478,...,15.7699,15.7699,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404
102981,2023-12-31 13:40:00,12.5549,12.5549,12.5549,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,...,15.7699,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404
102982,2023-12-31 13:45:00,12.5549,12.5549,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,...,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404
102983,2023-12-31 13:50:00,12.5549,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,...,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404


In [81]:
# add timestamp to the predictions
# Use appropriate timestamps from your DataFrame (e.g., 5-minute intervals)
timestamps = df.loc['2023-01-07 23:55:00':'2023-12-31 13:55:00'].index
# Create DataFrames for actual and predicted values

residual_pred = pd.DataFrame(y_pred_residual)
sced_time_stamp_local = actual_dap_rolled[['sced_time_stamp_local']]
residual_pred.insert(0, 'sced_time_stamp_local', sced_time_stamp_local)


In [83]:
residual_pred.columns = actual_dap_rolled.columns

In [85]:
pred_df = residual_pred.copy()

In [86]:
pred_df.iloc[:,1:] = residual_pred.iloc[:,1:] + actual_dap_rolled.iloc[:,1:]

In [89]:
pred_df.to_csv("pred_mlp_gaussian_residual")

In [98]:
# prepare the nodal real-time price in the required shape
actual = pd.read_csv('../../datasset/lmp_WAKEWE_ALL_2023.csv')
actual = actual.iloc[7*24*12 - 1:]
actual_rolled = pd.DataFrame([pd.Series(actual['price'].iloc[i+1:i+1+look_ahead].values) for i in range(len(actual) - look_ahead)])
actual_rolled.columns = [f'{i+1}' for i in range(look_ahead)]
actual_rolled['timestamp'] = actual['timestamp'].iloc[:len(actual_rolled)].values
actual_rolled = actual_rolled[['timestamp'] + [col for col in actual_rolled.columns if col != 'timestamp']]
actual_rolled = actual_rolled.iloc[:,:look_ahead+1]

In [99]:
actual_rolled

,timestamp,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,1/7/23 23:55,18.09,18.35,18.90,21.70,20.05,19.44,19.26,19.23,17.93,...,15.00,14.97,14.77,14.44,13.98,14.71,14.89,14.69,16.12,15.75
1,1/8/23 0:00,18.35,18.90,21.70,20.05,19.44,19.26,19.23,17.93,17.83,...,14.97,14.77,14.44,13.98,14.71,14.89,14.69,16.12,15.75,15.04
2,1/8/23 0:05,18.90,21.70,20.05,19.44,19.26,19.23,17.93,17.83,17.74,...,14.77,14.44,13.98,14.71,14.89,14.69,16.12,15.75,15.04,15.08
3,1/8/23 0:10,21.70,20.05,19.44,19.26,19.23,17.93,17.83,17.74,20.65,...,14.44,13.98,14.71,14.89,14.69,16.12,15.75,15.04,15.08,15.51
4,1/8/23 0:15,20.05,19.44,19.26,19.23,17.93,17.83,17.74,20.65,17.22,...,13.98,14.71,14.89,14.69,16.12,15.75,15.04,15.08,15.51,15.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102980,12/31/23 13:35,12.29,11.97,11.28,11.78,11.75,11.19,8.19,8.97,10.76,...,14.07,13.84,14.19,14.40,14.44,14.87,14.76,14.48,14.73,14.82
102981,12/31/23 13:40,11.97,11.28,11.78,11.75,11.19,8.19,8.97,10.76,11.24,...,13.84,14.19,14.40,14.44,14.87,14.76,14.48,14.73,14.82,14.54
102982,12/31/23 13:45,11.28,11.78,11.75,11.19,8.19,8.97,10.76,11.24,10.47,...,14.19,14.40,14.44,14.87,14.76,14.48,14.73,14.82,14.54,14.47
102983,12/31/23 13:50,11.78,11.75,11.19,8.19,8.97,10.76,11.24,10.47,7.73,...,14.40,14.44,14.87,14.76,14.48,14.73,14.82,14.54,14.47,14.39


In [90]:
# Storage parameters
bid_ahead = 1
horizon = int(look_ahead/12) - bid_ahead #in hours

Ts = 1/12  # hourly time step. If the price data is 5-min; Ts = 1/12. If it is hourly; Ts = 1
hr = int(1 / Ts)  # number of time steps in one hour

# Parameters
E = 1  # Energy rating in MWh
Pr = 1 / 2  # normalized power rating wrt energy rating
P = Pr * Ts  # actual power rating taking time step size into account
eta = 0.9  # efficiency
c = 10  # marginal discharge cost - degradation
initial_soc = 0.5

T = hr * horizon #the horizon as number of steps

In [ ]:
# optimization model
result_dfs = []

eS = np.zeros(T)  # initialize the SoC series
profit = np.zeros(T)  # initialize the profit series
revenue = np.zeros(T)  # initialize the revenue series

soc_var = cp.Variable(T, nonneg=True)
chr_var = cp.Variable(T, nonneg=True)
dis_var = cp.Variable(T, nonneg=True)
charge_decision = cp.Variable(T, boolean=True)
discharge_decision = cp.Variable(T, boolean=True)
discharge_active = cp.Variable(T, boolean=True)
M = 1e6  # A large constant
price_param = cp.Parameter(T)
soc_init_param = cp.Parameter()

cons = [
    soc_var[0] - soc_init_param == chr_var[0] * eta - dis_var[0] / eta,
    *[soc_var[i] - soc_var[i-1] == chr_var[i] * eta - dis_var[i] / eta for i in range(1, T)],
    *[chr_var[i] <= P for i in range(T)],
    *[soc_var[i] <= E for i in range(T)],
    *[charge_decision[i] + discharge_decision[i] <= 1 for i in range(T)],
    *[dis_var[i] <= discharge_active[i] * P for i in range(T)],
    *[dis_var[i] >= 0 for i in range(T)],
    *[price_param[i] >= -M * (1 - discharge_active[i]) for i in range(T)],
    *[price_param[i] <= M * discharge_active[i] for i in range(T)]
]

revenue_obj = cp.multiply(price_param, (dis_var - chr_var)) - c * dis_var
problem = cp.Problem(cp.Maximize(cp.sum(revenue_obj)), cons)


for start in tqdm(range(int(len(actual_rolled)-bid_ahead*12))):
    slice_actual = actual_rolled.iloc[start+bid_ahead*12, :T+1]
    slice_pred = pred_df.iloc[start, 1+bid_ahead*12:T+bid_ahead*12+1].copy()
    slice_pred.iloc[0] = slice_actual[1]  # Replace the first item with actual_rolled's first item

    price_param.value = slice_pred.values.astype(float)
    soc_init_param.value = initial_soc
    problem.solve(solver=cp.GUROBI)
    pS_dis = dis_var.value
    pS_chr = chr_var.value
    eS = soc_var.value
    revenue = slice_actual.iloc[1:].values.astype(float) * (pS_dis - pS_chr)
    profit = revenue - c * pS_dis
    storage_profile_sl = pd.Series({
        'time': slice_actual.iloc[0],
        'actual': slice_actual.iloc[1],
        'pred': slice_pred.iloc[0],
        'discharge': pS_dis[0],
        'charge': pS_chr[0],
        'soc': eS[0],
        'profit': profit[0],
        'revenue': revenue[0]})
    result_dfs.append(storage_profile_sl)
    initial_soc = eS[0]

final_result_lmp = pd.DataFrame(result_dfs)

#final_result_lmp.to_csv(str(bid_ahead)+'hr_bidding_results'+str(T)+'_lmp_5min_TimesFM_diff.csv', index=False)



  0%|          | 0/102973 [00:00<?, ?it/s]/tmp/ipykernel_2872676/3485764202.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slice_pred.iloc[0] = slice_actual[1]  # Replace the first item with actual_rolled's first item
  1%|▏         | 1440/102973 [00:18<20:00, 84.58it/s]

In [ ]:
final_result_lmp.profit.sum()

(104737, 96)


In [107]:
optimizer.slice_lmp_replace_sced()
print(optimizer.sced_reset.shape, sl_pred_reset.shape)

/insomnia001/home/hl3837/research/code/utils/storage_optimizer.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  lmp_data = pd.read_csv('../../datasset/lmp_WAKEWE_ALL_2023.csv', parse_dates=[0], index_col=0)


(104737, 97) (104737, 97)


# Storage Arbitrage
Perform storage arbitrage for WAKEWE_ALL node. Use the predicted system lambda to derive the best decisions to maximize profit.
The test year is 2023, so we need the nodal real-time price for that year.

In [ ]:
optimizer = StorageOptimizer(8)
optimizer.set_prediction_file(sl_pred) #"../../models/MLP/5min_sl_pred_dap_241h.csv"
#optimizer.slice_lmp_replace_sced()
final_result = optimizer.optimize()

sced shape is:  (104737, 97)
sced reset shape is:  (104737, 97)


In [ ]:
final_result.profit.sum()

np.float64(39730.464393017995)

In [ ]:
optimizer2 = StorageOptimizer(1)
optimizer2.set_prediction_file("../../models/MLP/5min_sl_actual_dap_241h.csv")
final_result2 = optimizer2.optimize()

100%|██████████| 104820/104820 [06:55<00:00, 252.55it/s]


In [ ]:
final_result2.profit.sum()

np.float64(58403.2749058563)